In [ ]:
import pandas as pd
import os
import gc
from tqdm import tqdm

In [ ]:
# Create a list to hold DataFrames
dfs = []
for root_dir, sub_dir, files in os.walk(r'G:/emr'):
    for file in tqdm(files, desc = 'Excel import', ncols = 100): # Wrap your iterable with tqdm()
        df = pd.read_excel(os.path.join(root_dir, file))
        dfs.append(df)
        gc.collect()  # Explicitly free memory
# Concatenate all DataFrames at once
result_df = pd.concat(dfs, ignore_index=True)

# Statistical variables

In [ ]:
result_df = pd.concat(dfs, ignore_index=True)
# Observations
print(len(result_df))

## Find the same patient

In [ ]:
# Initialize new columns for Name, Gender, and Age
result_df['Name'] = None
result_df['Gender'] = None
result_df['Age'] = None
result_df.reset_index(inplace=True, drop=True)
# Fill the new columns with values from XmlValue based on the content of XmlField
for idx, row in result_df.iterrows():
    if '姓名' in row['XmlField']:
        result_df.loc[idx, 'Name'] = row['XmlValue']
    elif '性别' in row['XmlField']:
        result_df.loc[idx, 'Gender'] = row['XmlValue']
    elif '年龄' in row['XmlField']:
        result_df.loc[idx, 'Age'] = row['XmlValue']

In [ ]:
for column in ['Name', 'Gender', 'Age']:
    result_df[column] = result_df.groupby('HospId')[column].transform('first')
# Initialize a new column for patient IDs
result_df['PatientID'] = -1
# Find the same patient using name, gender, and age
patient_cohort = result_df
unique_patients = patient_cohort.drop_duplicates(subset=['Name', 'Gender', 'Age'])
patient_ids = range(len(unique_patients))
    
patient_id_dict = {
    (row.Name, row.Gender, row.Age): id 
    for row, id in zip(unique_patients.itertuples(), patient_ids)
}
for row in result_df.itertuples():
    patient_info = (row.Name, row.Gender, row.Age)
    if patient_info in patient_id_dict:
        result_df.at[row.Index, 'PatientID'] = patient_id_dict[patient_info]
# Patient count
print(result_df['PatientID'].nunique())

In [ ]:
# Age convert
import re

def convert_age_to_years(age_str):
    # Check if input is not string
    if not isinstance(age_str, str):
        return age_str

    # Pattern for years, months, and days
    years_pattern = r'(\d+)\s*岁'
    months_pattern = r'(\d+)\s*个月'
    days_pattern = r'(\d+)\s*天'

    # Search for matches
    years_match = re.search(years_pattern, age_str)
    months_match = re.search(months_pattern, age_str)
    days_match = re.search(days_pattern, age_str)

    # Extract matched numbers and convert to float
    years = float(years_match.group(1)) if years_match else 0
    months = float(months_match.group(1)) / 12 if months_match else 0
    days = float(days_match.group(1)) / 365.25 if days_match else 0

    # Return sum of all (converted to years)
    return years + months + days

# Apply the function to the 'Age' column
result_df['Age'] = result_df['Age'].apply(convert_age_to_years)

In [ ]:
age_gender_name = result_df[['Name', 'Gender', 'Age', 'PatientID']]
age_gender_name = age_gender_name.drop_duplicates()
age_gender_name.reset_index(inplace=True, drop=True)

In [ ]:
# gender
print(age_gender_name['Gender'].value_counts())

In [ ]:
# age
print(age_gender_name.Age.describe())

In [ ]:
# age and gender statistics
male_EHR = age_gender_name.loc[age_gender_name['Gender'] == '男']
female_EHR = age_gender_name.loc[age_gender_name['Gender'] == '女']
male_infants_and_toddlers = male_EHR.loc[(2 >= male_EHR['Age']) & (male_EHR['Age'] > 0)]
female_infants_and_toddlers = female_EHR.loc[(2 >= female_EHR['Age']) & (female_EHR['Age'] > 0)]
male_preschool = male_EHR.loc[(4 >= male_EHR['Age']) & (male_EHR['Age'] > 2)]
female_preschool = female_EHR.loc[(4 >= female_EHR['Age']) & (female_EHR['Age'] > 2)]
male_school_age_children = male_EHR.loc[(12 >= male_EHR['Age']) & (male_EHR['Age'] > 4)]
female_school_age_children = female_EHR.loc[(12 >= female_EHR['Age']) & (female_EHR['Age'] > 4)]
male_teenagers = male_EHR.loc[(19 >= male_EHR['Age']) & (male_EHR['Age'] > 12)]
female_teenagers = female_EHR.loc[(19 >= female_EHR['Age']) & (female_EHR['Age'] > 12)]
male_young_adults = male_EHR.loc[(34 >= male_EHR['Age']) & (male_EHR['Age'] > 19)]
female_young_adults = female_EHR.loc[(34 >= female_EHR['Age']) & (female_EHR['Age'] > 29)]
male_adults = male_EHR.loc[(49 >= male_EHR['Age']) & (male_EHR['Age'] > 34)]
female_adults = female_EHR.loc[(49 >= female_EHR['Age']) & (female_EHR['Age'] > 34)]
male_middle_aged_adults = male_EHR.loc[(64 >= male_EHR['Age']) & (male_EHR['Age'] > 49)]
female_middle_aged_adults = female_EHR.loc[(64 >= female_EHR['Age']) & (female_EHR['Age'] > 49)]
male_seniors = male_EHR.loc[male_EHR['Age'] > 64]
female_seniors = female_EHR.loc[female_EHR['Age'] > 64]
print('The ratio of each age and gender groups are:\n' +
      'Male: \n'+
      'male_infants_and_toddlers: ' + str(len(male_infants_and_toddlers)/len(age_gender_name)) + '\n' +
      'male_preschool: ' + str(len(male_preschool)/len(age_gender_name))+'\n' +
      'male_school_age_children: ' + str(len(male_school_age_children)/len(age_gender_name))+'\n'+
      'male_teenagers: '+ str(len(male_teenagers)/len(age_gender_name))+'\n'+
      'male_young_adults: '+ str(len(male_young_adults)/len(age_gender_name))+'\n'+
      'male_adults: '+ str(len(male_adults)/len(age_gender_name))+'\n'+
      'male_middle_aged_adults: '+ str(len(male_middle_aged_adults)/len(age_gender_name))+'\n'+
      'male_seniors'+ str(len(male_seniors)/len(age_gender_name))+'\n'+
      'Female: \n'+
      'female_infants_and_toddlers: ' + str(len(female_infants_and_toddlers) / len(age_gender_name)) + '\n' +
      'female_preschool: ' + str(len(female_preschool) / len(age_gender_name)) + '\n' +
      'female_school_age_children: ' + str(len(female_school_age_children) / len(age_gender_name)) + '\n' +
      'female_teenagers: ' + str(len(female_teenagers) / len(age_gender_name)) + '\n' +
      'female_young_adults: ' + str(len(female_young_adults) / len(age_gender_name)) + '\n' +
      'female_adults: ' + str(len(female_adults) / len(age_gender_name)) + '\n' +
      'female_middle_aged_adults: ' + str(len(female_middle_aged_adults) / len(age_gender_name)) + '\n' +
      'female_seniors' + str(len(female_seniors) / len(age_gender_name)) + '\n')

## Search for misdiagnosis using empirical triggers

In [ ]:
# Perform the pivot operation
pivot_df = pd.pivot_table(result_df, index=['PatientID', 'Name', 'Gender', 'Age', 'OperTime', 'InEmrId'], columns='XmlField', values='XmlValue', aggfunc='first')
pivot_df['PatientID'] = pivot_df.index.get_level_values('PatientID')
pivot_df['Name'] = pivot_df.index.get_level_values('Name')
pivot_df['Gender'] = pivot_df.index.get_level_values('Gender')
pivot_df['Age'] = pivot_df.index.get_level_values('Age')
pivot_df['OperTime'] = pd.to_datetime(pivot_df.index.get_level_values('OperTime'))
pivot_df['InEmrId'] = pivot_df.index.get_level_values('InEmrId')
pivot_df.reset_index(inplace=True, drop=True)

In [ ]:
def misdiagnosis_label(df):
    df = df.sort_values(by=['PatientID', 'OperTime'])  # Sort the DataFrame by 'PatientID' and 'OperTime'
    df['TimeDiff'] = df.groupby('PatientID')['OperTime'].diff()  # Calculate time difference between consecutive readmissions
    df['Misdiag'] = ((df['TimeDiff'] >= pd.Timedelta(days=1)) & (df['TimeDiff'] <= pd.Timedelta(days=14))).astype(int)
    df['Misdiag'] = df['Misdiag'].shift(-1)
    df.at[df.index[-1], 'Misdiag'] = 0
    df = df.reset_index(drop=True)
    return df
labeled_misdiagnosis = misdiagnosis_label(pivot_df)

In [ ]:
column_name_chinese = pd.read_excel('G:/column_name_chinese.xlsx')
for col in labeled_misdiagnosis.columns:
    if col in column_name_chinese['字段名'].values:
        # get the English name corresponding to the Chinese name
        english_name = column_name_chinese.loc[column_name_chinese['字段名'] == col, '字段含义'].values[0]
        labeled_misdiagnosis.rename(columns={col: english_name}, inplace=True)

In [ ]:
# Compute proportion of null values in each column
null_proportion = labeled_misdiagnosis.isnull().mean()
# Get columns to keep
cols_to_keep = null_proportion[null_proportion <= 0.8].index
# Keep only the columns in cols_to_keep and drop the rest
labeled_misdiagnosis = labeled_misdiagnosis[cols_to_keep]
df_not_null = labeled_misdiagnosis[labeled_misdiagnosis['主诉内容'].notna()]
file_path = 'G:/labeled_misdiagnosis.csv'  # Path to save the CSV file
df_not_null.to_csv(file_path,sep=',',index=True,header=True, encoding='utf_8_sig')

## Plot correlation of patient readmission and diagnostic errors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [ ]:
# Convert TimeDiff to days
labeled_misdiagnosis['TimeDiff_days'] = labeled_misdiagnosis['TimeDiff'].dt.days

# Calculate misdiagnosis rate
labeled_misdiagnosis['Misdiag_rate'] = labeled_misdiagnosis.groupby('PatientID')['Misdiag'].transform('mean') * 100

# Define misdiagnosis rate bins
misdiag_rate_bins = np.linspace(0, 100, 11) # 0-100% in 5% increments

# Add a column to count readmissions per patient
labeled_misdiagnosis['Readmission_Count'] = labeled_misdiagnosis[labeled_misdiagnosis['TimeDiff_days']>0].groupby('PatientID')['TimeDiff_days'].transform('count')
# Define readmission count bins (you can modify this according to your data distribution)
readmission_count_bins = np.linspace(0, labeled_misdiagnosis['Readmission_Count'].max(), 21)

# Create a DataFrame with misdiagnosis rate, readmission count, and patient count for each bin
misdiag_subgroups = pd.DataFrame()
misdiag_subgroups['Misdiag_rate'] = pd.cut(labeled_misdiagnosis['Misdiag_rate'], bins=misdiag_rate_bins)
misdiag_subgroups['Readmission_Count'] = pd.cut(labeled_misdiagnosis['Readmission_Count'], bins=readmission_count_bins)
misdiag_subgroups['Patient_Count'] = labeled_misdiagnosis.groupby(['Misdiag_rate', 'Readmission_Count'])['PatientID'].transform('count')

# Remove duplicate rows (since each patient will have the same values for all three columns)
misdiag_subgroups.drop_duplicates(inplace=True)

# Find mean of 'Patient_Count' for each combination of 'Misdiag_rate' and 'Readmission_Count'
misdiag_subgroups = misdiag_subgroups.groupby(['Misdiag_rate', 'Readmission_Count'])['Patient_Count'].mean().reset_index()

# Pivot the data for the heatmap
pivot = misdiag_subgroups.pivot(index='Readmission_Count', columns='Misdiag_rate', values='Patient_Count')

# Generate the heatmap
fig, ax = plt.subplots(figsize=(5, 8))
cax = sns.heatmap(pivot, cmap="OrRd", linewidths=0.25, linecolor='black') # Add a gray border to each cell
plt.xticks([])
plt.yticks([])
plt.axis('off')
# Add a black border line to the heatmap
for _, spine in ax.spines.items():
    spine.set_visible(True)
    spine.set_color('black')
    spine.set_linewidth(0.25)

ax.invert_yaxis() # Make y-axis start at 0

# Save the figure
plt.savefig('./heatmap_count.png', dpi=1200)
plt.close()

In [ ]:
# Add a column to calculate the mean interval between readmissions per patient
labeled_misdiagnosis['Mean_Readmission_Interval'] = labeled_misdiagnosis[labeled_misdiagnosis['TimeDiff_days']>0].groupby('PatientID')['TimeDiff_days'].transform('mean')
# Define mean readmission interval bins (you can modify this according to your data distribution)
readmission_interval_bins = np.linspace(0, labeled_misdiagnosis['Mean_Readmission_Interval'].max(), 21)



# Create a DataFrame with misdiagnosis rate, readmission count, and patient count for each bin
misdiag_subgroups = pd.DataFrame()
misdiag_subgroups['Misdiag_rate'] = pd.cut(labeled_misdiagnosis['Misdiag_rate'], bins=misdiag_rate_bins)
misdiag_subgroups['Mean_Readmission_Interval'] = pd.cut(labeled_misdiagnosis['Mean_Readmission_Interval'], bins=readmission_interval_bins)
misdiag_subgroups['Patient_Count'] = labeled_misdiagnosis.groupby(['Misdiag_rate', 'Mean_Readmission_Interval'])['PatientID'].transform('count')

# Remove duplicate rows (since each patient will have the same values for all three columns)
misdiag_subgroups.drop_duplicates(inplace=True)

# Find mean of 'Patient_Count' for each combination of 'Misdiag_rate' and 'Readmission_Count'
misdiag_subgroups = misdiag_subgroups.groupby(['Misdiag_rate', 'Mean_Readmission_Interval'])['Patient_Count'].mean().reset_index()

# Pivot the data for the heatmap
pivot = misdiag_subgroups.pivot(index='Mean_Readmission_Interval', columns='Misdiag_rate', values='Patient_Count')

# Generate the heatmap
fig, ax = plt.subplots(figsize=(5, 8))
cax = sns.heatmap(pivot, cmap="YlGnBu", linewidths=0.25, linecolor='black') # Add a gray border to each cell
plt.xticks([])
plt.yticks([])
plt.axis('off')
# Add a black border line to the heatmap
for _, spine in ax.spines.items():
    spine.set_visible(True)
    spine.set_color('black')
    spine.set_linewidth(0.25)

ax.invert_yaxis() # Make y-axis start at 0

# Save the figure
plt.savefig('./heatmap_interval.png', dpi=1200)
plt.close

In [ ]:
# Count the number of patients per department and calculate proportions
department_counts = pivot_df['科室'].value_counts(normalize=True)

# Count the number of patients per ethnicity and calculate proportions
ethnicity_counts = pivot_df['民族'].value_counts(normalize=True)

import openpyxl
#with pd.ExcelWriter('G:/deparment_ethnicity.xlsx') as writer:  
    #department_counts.to_excel(writer, sheet_name='Department Counts')
    #ethnicity_counts.to_excel(writer, sheet_name='Ethnicity Counts')

In [ ]:
# Misdiagnosis ratio in derivation cohort
misdiag_percentage = (df_not_null['Misdiag'] == 1).mean() * 100
print(misdiag_percentage)
# EMRs in derivation cohort
len(labeled_misdiagnosis)

In [ ]:
df_copy = df_not_null.copy()

# Create a new DataFrame instead of a slice
selected_df = df_copy[['主诉内容', '传染病史', '其他病史', '心血管病史', '手术外伤史', '过敏史', '预防接种史', 'Misdiag']].copy()

# Fill NaNs in all involved columns
for col in ['主诉内容', '传染病史', '其他病史', '心血管病史', '手术外伤史', '过敏史', '预防接种史']:
    selected_df.loc[:, col] = selected_df.loc[:, col].fillna('')

# Combine the text columns
selected_df.loc[:, 'combined_text'] = selected_df['主诉内容'] + ' ' + selected_df['传染病史'] + ' ' + selected_df['其他病史'] + ' ' + selected_df['心血管病史'] + ' ' + selected_df['手术外伤史'] + ' ' + selected_df['过敏史'] + ' ' + selected_df['预防接种史']

In [ ]:
# Select only the 'combined_text' and 'Misdiag' columns
export_df = selected_df[['combined_text', 'Misdiag']]

# Export the DataFrame to a .txt file
export_df.to_csv('selected_df.txt', index=False, sep='\t')